In [7]:
import pandas as pd
import mysql.connector
import numpy as np

In [8]:
brfss_clean = pd.read_csv("brfss_processed.csv")

In [9]:
bool_map = {'Yes': 1, 'No': 0}
brfss_clean['smoked_100_cig'] = brfss_clean['smoked_100_cig'].map(bool_map)
brfss_clean['physical_activity'] = brfss_clean['physical_activity'].map(bool_map)
brfss_clean['overweight'] = brfss_clean['overweight'].map(bool_map)

In [10]:
brfss_clean = brfss_clean.replace({np.nan: None})

In [11]:
brfss_clean = brfss_clean[~brfss_clean['state'].isin(['Guam','Puerto Rico','Virgin Islands'])]

# dim_demographics

In [12]:
dim_demographics = brfss_clean[['sex', 'age', 'education', 'income', 'race']].drop_duplicates().copy()
dim_demographics['demographics_key'] = range(0, 11678)

In [13]:
dim_demographics = dim_demographics.rename(columns={
    'age': 'age_group',
    'education': 'education_level',
    'income': 'annual_income',
    'race': 'race_ethnicity'
})

In [14]:
demographics_values = dim_demographics[['demographics_key','sex', 'age_group', 'education_level', 'annual_income', 'race_ethnicity']].apply(tuple, axis=1).tolist()

In [15]:
demographics_insert_sql = """
INSERT INTO dim_demographics (demographics_key, sex, age_group, education_level, annual_income, race_ethnicity)
VALUES (%s, %s, %s, %s, %s,%s)
"""

In [16]:
conn = mysql.connector.connect(user = "root",
                                       password = "bowser140707",
                                       host = "localhost",
                                       database = "HealthCareDB")
cursor = conn.cursor()

cursor.executemany(demographics_insert_sql, demographics_values)

conn.commit()
cursor.close()
conn.close()

# dim_behaviors

In [17]:
dim_behaviors = brfss_clean[['smoked_100_cig', 'smoking_frequency', 'physical_activity', 'alcohol']].drop_duplicates().copy()
dim_behaviors['behaviors_key'] = range(0, 415)

In [18]:
dim_behaviors = dim_behaviors.rename(columns={
    'smoked_100_cig': 'smoked_100_lifetime',
    'physical_activity': 'reported_exercise',
    'alcohol': 'alcohol_use_frequency'
})

In [19]:
behaviors_values = dim_behaviors[['behaviors_key','smoked_100_lifetime', 'smoking_frequency', 'reported_exercise', 'alcohol_use_frequency']].apply(tuple, axis=1).tolist()

In [20]:
behaviors_insert_sql = """
INSERT INTO dim_behaviors
  (behaviors_key, smoked_100_lifetime, smoking_frequency, reported_exercise, alcohol_use_frequency)
VALUES (%s, %s, %s, %s, %s)
"""

In [21]:
conn = mysql.connector.connect(user = "root",
                                       password = "bowser140707",
                                       host = "localhost",
                                       database = "HealthCareDB")
cursor = conn.cursor()

cursor.executemany(behaviors_insert_sql, behaviors_values)

conn.commit()
cursor.close()
conn.close()

# dim_general_health

In [22]:
dim_general_health = brfss_clean[['gen_health', 'phys_health', 'mental_health']].drop_duplicates().copy()

In [23]:
dim_general_health['general_health_key'] = range(0, len(dim_general_health))

In [24]:
dim_general_health = dim_general_health.rename(columns={
    'gen_health':     'health_rating',
    'phys_health':    'days_physical_health_not_good',
    'mental_health':  'days_mental_health_not_good'
})

In [25]:
general_health_values = dim_general_health[
    ['general_health_key','health_rating', 'days_physical_health_not_good', 'days_mental_health_not_good']
].apply(tuple, axis=1).tolist()

In [26]:
general_health_insert_sql = """
INSERT INTO dim_general_health
  (general_health_key,health_rating, days_physical_health_not_good, days_mental_health_not_good)
VALUES (%s, %s, %s, %s)
"""

In [27]:
conn = mysql.connector.connect(user = "root",
                                       password = "bowser140707",
                                       host = "localhost",
                                       database = "HealthCareDB")
cursor = conn.cursor()

cursor.executemany(general_health_insert_sql, general_health_values)

conn.commit()
cursor.close()
conn.close()

# dim_checkup_engagement

In [28]:
dim_checkup_engagement = brfss_clean[['cholesterol_check', 'routine_checkup']].drop_duplicates().copy()
dim_checkup_engagement['checkup_engagement_key'] = range(0, len(dim_checkup_engagement))

In [29]:
dim_checkup_engagement = dim_checkup_engagement.rename(columns={
    'cholesterol_check': 'time_since_cholcheck',
    'routine_checkup':   'time_since_gencheck'
})

In [30]:
checkup_engagement_values = dim_checkup_engagement[
    ['checkup_engagement_key', 'time_since_cholcheck', 'time_since_gencheck']
].apply(tuple, axis=1).tolist()

In [31]:
checkup_engagement_insert_sql = """
INSERT INTO dim_checkup_engagement
  (checkup_engagement_key, time_since_cholcheck, time_since_gencheck)
VALUES (%s, %s, %s)
"""

In [32]:
conn = mysql.connector.connect(user = "root",
                                       password = "bowser140707",
                                       host = "localhost",
                                       database = "HealthCareDB")
cursor = conn.cursor()

cursor.executemany(checkup_engagement_insert_sql, checkup_engagement_values)

conn.commit()
cursor.close()
conn.close()

# dim_bmi

In [33]:
dim_BMI = brfss_clean[['bmi', 'bmi_cat', 'overweight']].drop_duplicates().copy()
dim_BMI['bmi_key'] = range(0, len(dim_BMI))

In [34]:
dim_BMI = dim_BMI.rename(columns={
    'bmi':            'bmi_value',
    'bmi_cat':        'bmi_category',
    'overweight':     'overweight_obese'
})

In [35]:
bmi_values = dim_BMI[
    ['bmi_key','bmi_value', 'bmi_category', 'overweight_obese']
].apply(tuple, axis=1).tolist()


In [36]:
bmi_insert_sql = """
INSERT INTO dim_BMI
  (bmi_key, bmi_value, bmi_category, overweight_obese)
VALUES (%s, %s, %s, %s)
"""

In [37]:
conn = mysql.connector.connect(user = "root",
                                       password = "bowser140707",
                                       host = "localhost",
                                       database = "HealthCareDB")
cursor = conn.cursor()

cursor.executemany(bmi_insert_sql, bmi_values)

conn.commit()
cursor.close()
conn.close()

# dim_state

In [38]:
chrr = pd.read_csv("chr_processed.csv")
eco = pd.read_csv("state.csv")

In [39]:
chrr['key'] = range(0, 51)

In [40]:
chrr_eco_mapped = eco.merge(
    chrr[['key', 'state']],
    left_on='State', right_on='state',
    how='left'
)

In [41]:
pop_df = chrr_eco_mapped[['key', 'Population']].drop_duplicates().copy()
chrr = chrr.merge(pop_df, on='key', how='left')

In [42]:
chrr = chrr.replace({np.nan: None})

In [43]:
state_values = chrr[
    ['key',
        'state',
    'Population']
].apply(tuple, axis=1).tolist()

In [44]:
state_insert_sql = """
INSERT INTO dim_state
  (state_key, state_name,population)
VALUES (%s, %s, %s)
"""

In [45]:
conn = mysql.connector.connect(user = "root",
                                       password = "bowser140707",
                                       host = "localhost",
                                       database = "HealthCareDB")
cursor = conn.cursor()

cursor.executemany(state_insert_sql, state_values)

conn.commit()
cursor.close()
conn.close()

# dim_state_env

In [46]:
state_env_values = chrr[
    ['key',
    'avg_pm25',
    'water_violation_rate',
    'food_env_index',
    'exercise_access',
    'limited_healthyfood_access']
].apply(tuple, axis=1).tolist()


In [47]:
state_env_insert_sql = """
INSERT INTO dim_state_env
  (state_key, pm25_level, water_violation_rate, food_env_index, exercise_access, limited_healthyfood_access)
VALUES (%s, %s, %s, %s, %s, %s)
"""

In [48]:
conn = mysql.connector.connect(user = "root",
                                       password = "bowser140707",
                                       host = "localhost",
                                       database = "HealthCareDB")
cursor = conn.cursor()

cursor.executemany(state_env_insert_sql, state_env_values)

conn.commit()
cursor.close()
conn.close()

# dim_eco

In [49]:
state_eco_values = chrr_eco_mapped[
    ['key',
    'GDP (in billions USD)',
    'Insurance_Cost_Monthly_USD']
].apply(tuple, axis=1).tolist()


In [50]:
state_eco_insert_sql = """
INSERT INTO dim_state_eco
  (state_key, gdp, insurance_cost)
VALUES (%s, %s, %s)
"""

In [51]:
conn = mysql.connector.connect(user = "root",
                                       password = "bowser140707",
                                       host = "localhost",
                                       database = "HealthCareDB")
cursor = conn.cursor()

cursor.executemany(state_eco_insert_sql, state_eco_values)

conn.commit()
cursor.close()
conn.close()

# fact table

In [52]:
fact_df = brfss_clean.reset_index(drop=True).copy()

In [53]:
fact_df['respondent_id'] = range(0, 425106)

In [54]:
fact_df = fact_df.rename(columns={
    'age':'age_group',
    'education':'education_level',
    'income':'annual_income',
    'race':'race_ethnicity'
})
fact_df = fact_df.merge(
    dim_demographics[['demographics_key','sex','age_group','education_level','annual_income','race_ethnicity']],
    on=['sex','age_group','education_level','annual_income','race_ethnicity'],
    how='left'
)

In [ ]:
fact_df = fact_df.rename(columns={
    'smoked_100_cig':'smoked_100_lifetime',
    'physical_activity':'reported_exercise',
    'alcohol':'alcohol_use_frequency'
})


fact_df = fact_df.merge(
    dim_behaviors[['behaviors_key','smoked_100_lifetime','smoking_frequency','reported_exercise','alcohol_use_frequency']],
    on=['smoked_100_lifetime','smoking_frequency','reported_exercise','alcohol_use_frequency'],
    how='left'
)

In [ ]:
fact_df = fact_df.rename(columns={
    'gen_health':'health_rating',
    'phys_health':'days_physical_health_not_good',
    'mental_health':'days_mental_health_not_good'
})
fact_df = fact_df.merge(
    dim_general_health[['general_health_key','health_rating','days_physical_health_not_good','days_mental_health_not_good']],
    on=['health_rating','days_physical_health_not_good','days_mental_health_not_good'],
    how='left'
)


In [ ]:
fact_df = fact_df.rename(columns={
    'cholesterol_check':'time_since_cholcheck',
    'routine_checkup':'time_since_gencheck'
})
fact_df = fact_df.merge(
    dim_checkup_engagement[['checkup_engagement_key','time_since_cholcheck','time_since_gencheck']],
    on=['time_since_cholcheck','time_since_gencheck'],
    how='left'
)

In [ ]:
fact_df = fact_df.rename(columns={
    'bmi':'bmi_value',
    'bmi_cat':'bmi_category',
    'overweight':'overweight_obese'
})

fact_df = fact_df.merge(
    dim_BMI[['bmi_key','bmi_value','bmi_category','overweight_obese']],
    on=['bmi_value','bmi_category','overweight_obese'],
    how='left'
)

In [ ]:
dim_state = pd.read_csv('dim_state.csv')

In [ ]:
fact_df = fact_df.merge(
    dim_state, 
    left_on='state', 
    right_on='state_name', 
    how='left'
)

In [ ]:
fact_heart_health = fact_df[
    ['respondent_id',
     'heart_disease',
     'heart_attack',
     'high_blood_pressure','diabetes','stroke',
     'demographics_key','behaviors_key','general_health_key',
     'checkup_engagement_key','bmi_key','state_key']
].rename(columns={
    'heart_disease':'heart_disease_diagnosis',
    'heart_attack': 'heart_attack_diagnosis',
    'high_blood_pressure':'high_blood_pressure_diagnosis',
    'diabetes':'diabetes_diagnosis',
    'stroke':'stroke_diagnosis'
})

In [ ]:
heart_health_values = fact_heart_health.apply(tuple, axis=1).tolist()

In [ ]:
heart_health_insert_sql = """
INSERT INTO fact_heart_health
  (respondent_id,
   heart_disease_diagnosis,
   heart_attack_diagnosis,
   high_blood_pressure_diagnosis,
   diabetes_diagnosis,
   stroke_diagnosis,
   demographics_key,
   behaviors_key,
   general_health_key,
   checkup_engagement_key,
   bmi_key,
   state_key)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [ ]:
conn = mysql.connector.connect(user = "root",
                                       password = "bowser140707",
                                       host = "localhost",
                                       database = "HealthCareDB")
cursor = conn.cursor()

cursor.executemany(heart_health_insert_sql, heart_health_values)

conn.commit()
cursor.close()
conn.close()